<a href="https://colab.research.google.com/github/klordo/nlp_homeworks/blob/hw1/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

До этого не работал в колабе. Читал правила красивого написания для ноутбуков. Как я понял, эти правила для проектов-ноутбуков, но отчасти не для колаба. Потому установки зависимостей делал перед каждым инструментом. Могу исправить и переместить вверх.

Пробовал различные инструменты, оставил два варианта:
Первый через pymorphy2 и nltk без удаления стоп слов.
Второй через spacy.
Не решил одну проблему, о которой написал в самом конце.

Если нужны будут исправления, то хотел бы доделать до максимального балла, если это возможно)

In [ ]:
!pip install corus

In [ ]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.1/lenta-ru-news.csv.bz2

In [3]:
from corus import load_lenta2

path = 'lenta-ru-news.csv.bz2'
records = load_lenta2(path)

In [4]:
dataset = [next(records).text for i in range(1000)]
dataset[0]

'Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии. «Русский инвалид», 16 сентября 1914 года.'

In [ ]:
!pip install clean-text

In [6]:
from cleantext import replace_emails, replace_numbers, replace_urls

Добавил пробелы, потому что слова ниже были иногда присоединенные вплотную без пробела к другим словам.
Хотел добавить символы "<>", но они образовывали отдельные токены.

In [7]:
dataset_clean = dataset.copy()
dataset_clean = [replace_emails(text, ' email ') for text in dataset_clean]
dataset_clean = [replace_numbers(text, ' number ') for text in dataset_clean]
dataset_clean = [replace_urls(text, ' url ') for text in dataset_clean]

In [8]:
import re

In [9]:
symbols = set()
for text in dataset_clean:
    for char in text:
        if not char.isalpha() and not char.isdigit() and char not in ' .,!?':
            symbols.add(char)
symbols

{'"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 '-',
 '/',
 ':',
 ';',
 '=',
 '\xa0',
 '«',
 '»',
 '–',
 '—',
 '’',
 '…',
 '№'}

In [10]:
pattern = r'[^\w\s\d.,!?]+'
dataset_clean = [re.sub(pattern, '', text) for text in dataset_clean]

In [11]:
dataset_clean[0]

'Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра  number  сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к Саноку, обстреливалась с высот нашей батареей и бежала, бросив парки, обоз и автомобили. Вылазки гарнизона Перемышля остаются безуспешными. При продолжающемся отступлении австрийцев обнаруживается полное перемешивание их частей, захватываются новые партии пленных, орудия и прочая материальная часть. На перевале Ужок мы разбили неприятельский отряд, взяли его артиллерию и много пленных и, продолжая преследовать, вступили в пределы Венгрии. Русский инвалид,  number  сентября  number  года.'

nltk и pymorphy2

In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
from nltk.tokenize import (
    sent_tokenize,
    word_tokenize,
)

Данным инструментом решил разделить список документов по спискам предложений, каждый из которых делится на списки слов.

In [14]:
dataset_tokenized = [[word_tokenize(sent, language='russian') for sent in sent_tokenize(text, language='russian')] for text in dataset_clean]
dataset_tokenized[0][:2]

[['Бои',
  'у',
  'Сопоцкина',
  'и',
  'Друскеник',
  'закончились',
  'отступлением',
  'германцев',
  '.'],
 ['Неприятель',
  ',',
  'приблизившись',
  'с',
  'севера',
  'к',
  'Осовцу',
  'начал',
  'артиллерийскую',
  'борьбу',
  'с',
  'крепостью',
  '.']]

С помощью pymorphy2 провел лемматизацию без удаления стоп слов.

In [ ]:
!pip install pymorphy2

In [16]:
import pymorphy2

In [17]:
morph = pymorphy2.MorphAnalyzer()

In [18]:
dataset_lemmatized = [[[morph.parse(word)[0].normal_form for word in sent] for sent in text] for text in dataset_tokenized]

In [19]:
dataset_lemmatized[0][:2]

[['бой',
  'у',
  'сопоцкина',
  'и',
  'друскеник',
  'закончиться',
  'отступление',
  'германец',
  '.'],
 ['неприятель',
  ',',
  'приблизиться',
  'с',
  'север',
  'к',
  'осовца',
  'начать',
  'артиллерийский',
  'борьба',
  'с',
  'крепость',
  '.']]

Вариант с помощью spacy для токенизации, лемматизации и удаления стоп слов.

In [20]:
import spacy

In [ ]:
!python3 -m spacy download ru_core_news_sm

In [22]:
nlp = spacy.load("ru_core_news_sm")

In [23]:
dataset_doc = [[token.lemma_ for token in nlp(doc) if not token.is_stop] for doc in dataset_clean]

In [24]:
dataset_doc[0][:10]

['бой',
 'сопоцкина',
 'друскеник',
 'закончиться',
 'отступление',
 'германец',
 '.',
 'неприятель',
 ',',
 'приблизиться']

In [25]:
import re

In [26]:
bad_tokens = set()
for i, text in enumerate(dataset_doc):
    for word in text:
        if re.search(r'[^a-zA-Zа-яА-ЯёЁ]', word) and word not in '.,!?':
            bad_tokens.add(word)

list(bad_tokens)[:20]

['.com',
 '..',
 'существования,340',
 'т.е.',
 '12сентября',
 'w.t.f.r.c.',
 'января1999',
 '.00закрываются',
 'с.петербурга',
 'в.матвиенко',
 '.Ru',
 'id3',
 'декабря1998',
 '26полнометражных',
 '2428сделки',
 'ст.',
 'в.у.',
 'г.',
 'сб.',
 'п.']

Решил убрать плохие токены. Либо детально очищать текст, либо потерять часть информации. Я выбрал второе. Таких токенов по сравнению с размером датасета в целом не так много.

In [27]:
dataset_doc = [[word for word in text if word not in bad_tokens] for text in dataset_doc]

Не решенная проблема - многие слова не разделены пробелами.

In [28]:
for doc in dataset_doc[:10]:
  for token in doc:
    if len(token) > 20:
      print(token)

фотографкорреспондент
воротамкенсингтонского
сдержаннойпоминальной
оглашеныокончательные
французскиеследователи
обвиненийфотографампапарацци


In [29]:
!pip install wordninja

In [30]:
import wordninja

In [31]:
text = 'этоттекстнеразделенпробелами'
words = wordninja.split(text)
words

[]

Необходимо перевести модель на русскую, но я пока не нашел способ как ее найти и скачать в колаб...

Как я понял, необходимо для этого найти руский словарь.